In [15]:
import warnings 
warnings.simplefilter("ignore")
import numpy as np
import pandas as pd
#from statsforecast.core import StatsForecast
#from statsforecast.models import AutoARIMA, Naive
from hierarchicalforecast.core import HierarchicalReconciliation
#from hierarchicalforecast.evaluation import evaluate
from hierarchicalforecast.methods import BottomUp, TopDown, MinTrace, TopDownSparse
#from utilsforecast.losses import mseaoaoa

In [16]:
S = pd.read_pickle("S_df.pkl")
tags = pd.read_pickle("tags.bin")
lp = {"l1" :(0,1),"l2" :(1,(1+3)),"l4" :((1+3),(1+3+3)),"l3" :((1+3+3),(1+3+3+10)),"l5" :((1+3+10+3),(1+3+10+3+7)),
      "l6" :((1+3+10+3+7),(1+3+10+3+7+9)),"l7" :((1+3+10+3+7+9),(1+3+10+3+7+9+21)),"l8" :((1+3+10+3+7+9+21),(1+3+10+3+7+9+21+30)),
      "l10":((1+3+10+3+7+9+21+30),(1+3+10+3+7+9+21+30+3049)),"l9" :((1+3+10+3+7+9+21+30+3049),(1+3+10+3+7+9+21+30+3049+70)),
      "l11":((1+3+10+3+7+9+21+30+70+3049),(1+3+10+3+7+9+21+30+70+3049+9147)),"l12":((1+3+10+3+7+9+21+30+70+3049+9147),(1+3+10+3+7+9+21+30+70+3049+9147+30490))}
lvls = []
#n = 1
c1 = 1
c2 = 1941

lvls = []
for i in lp.keys():
    a,b = lp[i]
    lvls.append([i[1:]]*c1*(b-a))
   # n+=1
levels_c1 = pd.concat([pd.DataFrame({'levels':lvls[i]}) for i in range(len(lvls))]).reset_index(drop=True)

lvls = []
for i in lp.keys():
    a,b = lp[i]
    lvls.append([i[1:]]*c2*(b-a))
   # n+=1
levels_c2 = pd.concat([pd.DataFrame({'levels':lvls[i]}) for i in range(len(lvls))]).reset_index(drop=True)

d_c1 = levels_c1.iloc[:,0]
d_c2 = levels_c2.iloc[:,0]
#d_c3 = levels_c3.iloc[:,0]
#d_c4 = levels_c4.iloc[:,0]

idx_c1 = d_c1[d_c1.isin(["10","11","12"])].index
idx_c2 = d_c2[d_c2.isin(["10","11","12"])].index
#idx_c3 = d_c3[d_c3.isin(["10","11","12"])].index
#idx_c4 = d_c4[d_c4.isin(["10","11","12"])].index

S = S.iloc[idx_c1,:]
S.to_pickle('721S_df.pkl')

In [17]:
%%time
path  = 'D:/yjz/Extension for hts/JayCode/Data/'

Y_ets_tr  = pd.read_pickle("h1_AutoETSfitt.pkl")
Y_ets_ts = pd.read_pickle("h1_AutoETSbase.pkl")
uidds_ts = pd.read_pickle("h1_AutoETSbase.pkl")[["unique_id",'ds']]
uidds_tr = pd.read_pickle("h1_AutoETSfitt.pkl")[["unique_id",'ds','y']]

Y_lgbm_ts = pd.concat([uidds_ts.reset_index(drop=True),
                       pd.read_pickle("h1_LGBMbase_recursive.pkl").reset_index(drop=True)],axis = 1)

Y_lgbm_tr = pd.concat([uidds_tr.reset_index(drop=True), 
                       pd.read_pickle("h1_LGBMfitt.pkl")],axis=1)

Y_lgbm_tr.columns = ["unique_id",'ds','y','LGBMRegressor']
Y_lgbm_ts.columns = ["unique_id",'ds','LGBMRegressor']

lis = S.iloc[:,0].unique()

Y_ets_tr  = Y_ets_tr.loc[Y_ets_tr["unique_id"].isin(lis),:]
Y_lgbm_tr = Y_lgbm_tr.loc[Y_lgbm_tr["unique_id"].isin(lis),:]

Y_ets_ts  = Y_ets_ts.loc[Y_ets_ts["unique_id"].isin(lis),:]
Y_lgbm_ts = Y_lgbm_ts.loc[Y_lgbm_ts["unique_id"].isin(lis),:]

Wall time: 28.7 s


In [18]:
%%time
y = pd.read_pickle("etsdata.pkl")
y = pd.concat([levels_c2,y],axis=1)
y = y.loc[y['unique_id'].isin(lis),:]
y1 = y.copy()
y['unique_id'] = pd.Categorical(y['unique_id'], categories=lis, ordered=True)
y = y.sort_values(['unique_id','ds'])
y_hist  = y[y['ds']<=1913]
y_ts2   = y[y['ds']>1913]

Wall time: 1min 8s


In [5]:
y_ts2

,levels,unique_id,ds,y
29705036,12,TOTAL/FOODS/FOODS_1/FOODS_1_001,1914,4
29705037,12,TOTAL/FOODS/FOODS_1/FOODS_1_001,1915,5
29705038,12,TOTAL/FOODS/FOODS_1/FOODS_1_001,1916,7
29705039,12,TOTAL/FOODS/FOODS_1/FOODS_1_001,1917,1
29705040,12,TOTAL/FOODS/FOODS_1/FOODS_1_001,1918,9
...,...,...,...,...
83152435,12,TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_...,1937,0
83152436,12,TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_...,1938,0
83152437,12,TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_...,1939,0
83152438,12,TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_...,1940,0


# BU, TDFP AND MINTRACE

### BU

In [19]:
%%time
# Reconcile the base predictions
bottomup = [
    BottomUp()
    #TopDown("proportion_averages"),#'average_proportion'
    #MinTrace("mint_shrink",nonnegative = True )
]
bu = HierarchicalReconciliation(reconcilers=bottomup)
#hrec.fit(Y_df = Y_df, Y_hat_df = Y_hat_df)
h1_lgb_bu = bu.reconcile(Y_hat_df=Y_lgbm_ts, S=S, tags=tags)#,Y_df=Y_lgbm_tr)
h1_ets_bu = bu.reconcile(Y_hat_df=Y_ets_ts, S=S, tags=tags)#,Y_df=Y_ets_tr)

tr_lgb_bu = bu.reconcile(Y_hat_df=Y_lgbm_tr, S=S, tags=tags)
tr_ets_bu = bu.reconcile(Y_hat_df=Y_ets_tr, S=S, tags=tags)

tr_y = bu.reconcile(Y_hat_df=y_hist[['unique_id','ds','y']], S=S, tags=tags)
ts_y = bu.reconcile(Y_hat_df=y_ts2[['unique_id','ds','y']], S=S, tags=tags)

Wall time: 14min 32s


In [20]:
h1lgb_bs = h1_lgb_bu['LGBMRegressor']
h1ets_bs = h1_ets_bu['AutoETS']
h1ets_bs.to_pickle("721latest_etsbase.pkl")
h1lgb_bs.to_pickle("721latest_lgbbase.pkl")

h1lgb_bu = h1_lgb_bu['LGBMRegressor/BottomUp']
h1ets_bu = h1_ets_bu['AutoETS/BottomUp']

ts_y.to_pickle("721future_28.pkl")
tr_y.to_pickle("721past_1913.pkl")
# trainning bottom
tr_lbu = tr_lgb_bu['LGBMRegressor/BottomUp']
tr_ebu = tr_ets_bu['AutoETS/BottomUp']
tr_lbs = tr_lgb_bu["LGBMRegressor"]
tr_ebs = tr_ets_bu["AutoETS"]

#del ts_y, tr_y,h1_lgb_bu, h1_ets_bu

### TDFP：

In [21]:
l10_12 = ['total/cat_id/dept_id/item_id', 'total/state_id/cat_id/dept_id/item_id', 'total/state_id/store_id/cat_id/dept_id/item_id']
tags_ = {i:tags[i] for i in l10_12}
ts  = pd.concat([levels_c1,y_ts2.iloc[:,:2]])
ts2 = y_ts2[y_ts2['levels'].isin(['10','11','12'])]
prods = [i.split('/')[-1] for i in tags_[l10_12[0]]]
prods

['FOODS_1_001',
 'FOODS_1_002',
 'FOODS_1_003',
 'FOODS_1_004',
 'FOODS_1_005',
 'FOODS_1_006',
 'FOODS_1_008',
 'FOODS_1_009',
 'FOODS_1_010',
 'FOODS_1_011',
 'FOODS_1_012',
 'FOODS_1_013',
 'FOODS_1_014',
 'FOODS_1_015',
 'FOODS_1_016',
 'FOODS_1_017',
 'FOODS_1_018',
 'FOODS_1_019',
 'FOODS_1_020',
 'FOODS_1_021',
 'FOODS_1_022',
 'FOODS_1_023',
 'FOODS_1_024',
 'FOODS_1_025',
 'FOODS_1_026',
 'FOODS_1_027',
 'FOODS_1_028',
 'FOODS_1_029',
 'FOODS_1_030',
 'FOODS_1_031',
 'FOODS_1_032',
 'FOODS_1_033',
 'FOODS_1_034',
 'FOODS_1_035',
 'FOODS_1_036',
 'FOODS_1_037',
 'FOODS_1_038',
 'FOODS_1_039',
 'FOODS_1_040',
 'FOODS_1_041',
 'FOODS_1_042',
 'FOODS_1_043',
 'FOODS_1_044',
 'FOODS_1_045',
 'FOODS_1_046',
 'FOODS_1_047',
 'FOODS_1_048',
 'FOODS_1_049',
 'FOODS_1_050',
 'FOODS_1_051',
 'FOODS_1_052',
 'FOODS_1_053',
 'FOODS_1_054',
 'FOODS_1_055',
 'FOODS_1_056',
 'FOODS_1_057',
 'FOODS_1_058',
 'FOODS_1_059',
 'FOODS_1_060',
 'FOODS_1_061',
 'FOODS_1_062',
 'FOODS_1_063',
 'FOODS_

In [22]:
%%time
from tqdm import tqdm
tdfp = [TopDown("forecast_proportions")]
tdfp = HierarchicalReconciliation(reconcilers=tdfp)

def tt(df_,prods,s, tags_):
    orfcst = []
    for j in tqdm(prods):
        b = df_[(df_['unique_id'].str.contains(j))]
        #idx = b.index
        bs = s[(s['unique_id'].str.contains(j))]
        bss = pd.concat([bs['unique_id'],bs.loc[:,bs.columns.str.contains(j)]],axis=1)
        ortdfp = tdfp.reconcile(Y_hat_df=b, S=bss, tags=tags_)
        #ortdfp['idx'] = idx
        orfcst.append(ortdfp)
    return pd.concat(orfcst)


#hrec.fit(Y_df = Y_df, Y_hat_df = Y_hat_df)
h1_lgb_td = tt(df_= Y_lgbm_ts, 
               prods = prods,
               s = S,
               tags_ = tags_) #tdfp.reconcile(Y_hat_df=Y_lgbm_ts, S=S, tags=tags)#,Y_df=Y_lgbm_tr)
h1_ets_td = tt(df_= Y_ets_ts, 
               prods = prods,
               s = S,
               tags_ = tags_)#h1_ets_td =  #tdfp.reconcile(Y_hat_df=Y_ets_ts, S=S, tags=tags)#,Y_df=Y_ets_tr)
'''
tr_lgb_td = tdfp.reconcile(Y_hat_df=Y_lgbm_tr, S=S, tags=tags)
#tt(df_= Y_lgbm_tr, 
               #prods = prods,
               #s = S,
               #tags_ = tags_)#tdfp.reconcile(Y_hat_df=Y_lgbm_tr, S=S, tags=tags)
tr_ets_td = tdfp.reconcile(Y_hat_df=Y_ets_tr, S=S, tags=tags)


tt(df_= Y_ets_tr, 
               prods = prods,
               s = S,
               tags_ = tags_)
'''
#h1lgb_td = h1_lgb_td.iloc[:,-1]
#h1ets_td = h1_ets_td.iloc[:,-1]
#tr_ltd = tr_lgb_td.iloc[:,-1]
#tr_etd = tr_ets_td.iloc[:,-1]


#tr_y = tdfp.reconcile(Y_hat_df=y_hist, S=S, tags=tags)
#ts_y = tdfp.reconcile(Y_hat_df=y_ts2, S=S, tags=tags)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3049/3049 [27:25<00:00,  1.85it/s]


Wall time: 54min 44s


'\ntr_lgb_td = tdfp.reconcile(Y_hat_df=Y_lgbm_tr, S=S, tags=tags)\n#tt(df_= Y_lgbm_tr, \n               #prods = prods,\n               #s = S,\n               #tags_ = tags_)#tdfp.reconcile(Y_hat_df=Y_lgbm_tr, S=S, tags=tags)\ntr_ets_td = tdfp.reconcile(Y_hat_df=Y_ets_tr, S=S, tags=tags)\n\n\ntt(df_= Y_ets_tr, \n               prods = prods,\n               s = S,\n               tags_ = tags_)\n'

In [23]:
h1_ets_td['unique_id'] = pd.Categorical(h1_ets_td['unique_id'], categories=lis, ordered=True)
h1_lgb_td['unique_id'] = pd.Categorical(h1_lgb_td['unique_id'], categories=lis, ordered=True)

h1_ets_td = h1_ets_td.sort_values(['unique_id','ds'])
h1_lgb_td = h1_lgb_td.sort_values(['unique_id','ds'])
'''
tr_ets_td['unique_id'] = pd.Categorical(tr_ets_td['unique_id'], categories=lis, ordered=True)
tr_lgb_td['unique_id'] = pd.Categorical(tr_lgb_td['unique_id'], categories=lis, ordered=True)

tr_ets_td = tr_ets_td.sort_values(['unique_id','ds'])
tr_lgb_td = tr_lgb_td.sort_values(['unique_id','ds'])'''

"\ntr_ets_td['unique_id'] = pd.Categorical(tr_ets_td['unique_id'], categories=lis, ordered=True)\ntr_lgb_td['unique_id'] = pd.Categorical(tr_lgb_td['unique_id'], categories=lis, ordered=True)\n\ntr_ets_td = tr_ets_td.sort_values(['unique_id','ds'])\ntr_lgb_td = tr_lgb_td.sort_values(['unique_id','ds'])"

In [24]:
h1lgb_td = h1_lgb_td.iloc[:,-1]
h1ets_td = h1_ets_td.iloc[:,-1]
#tr_ltd = tr_lgb_td.iloc[:,-1]
#tr_etd = tr_ets_td.iloc[:,-1]

In [ ]:
h1_ets_td['unique_id'].unique()

### Mint

In [25]:
%%time
# Reconcile the base predictions
mintrace = [
    #BottomUp(),
    #TopDown("proportion_averages"),#'average_proportion'
    MinTrace("mint_shrink" )
]
mint = HierarchicalReconciliation(reconcilers=mintrace)
h1_lgb_mint = mint.reconcile(Y_hat_df= Y_lgbm_ts, S=S, tags=tags, Y_df=Y_lgbm_tr)
h1_ets_mint = mint.reconcile(Y_hat_df= Y_ets_ts,  S=S, tags=tags, Y_df=Y_ets_tr)
h1lgb_mint = h1_lgb_mint.iloc[:,-1]
h1ets_mint = h1_ets_mint.iloc[:,-1]

tr_lgb_mint = mint.reconcile(Y_hat_df= Y_lgbm_tr[['unique_id','ds','LGBMRegressor']],
                             S=S, tags=tags, Y_df=Y_lgbm_tr)
tr_ets_mint = mint.reconcile(Y_hat_df= Y_ets_tr[['unique_id','ds','AutoETS']], 
                             S=S, tags=tags, Y_df=Y_ets_tr)

tr_lmint = tr_lgb_mint.iloc[:,-1]
tr_emint = tr_ets_mint.iloc[:,-1]

#del h1_lgb_mint,h1_ets_mint

Wall time: 1h 1min 53s


# Saver

In [14]:
h1lgb_td

0      5.689794
1      4.679130
2      4.798928
3      5.980217
4      5.048564
         ...   
387    0.108058
388    0.105750
389    0.095396
390    0.180732
391    0.110571
Name: LGBMRegressor/TopDown_method-forecast_proportions, Length: 1195208, dtype: float64

In [26]:
#h1_ets_td = topdown(y, h1ets_ts, td_l = [])
#h1_lgb_td = topdown(y, h1lgb_ts, td_l = [])

allfcst = pd.DataFrame({"lgb_base": h1lgb_bs,
                        "lgb_bu":   h1lgb_bu,
                        "lgb_td":   h1lgb_td.reset_index(drop=True),
                        "lgb_mint": h1lgb_mint,
                        "ets_base": h1ets_bs,
                        "ets_bu":   h1ets_bu,
                        "ets_td":   h1ets_td.reset_index(drop=True),
                        "ets_mint": h1ets_mint})
#allfcst.to_pickle("fcsts_base2cohe.pkl")

allfitt = pd.DataFrame({"lgb_base": tr_lbs,
                        "lgb_bu":   tr_lbu,
                        "lgb_td":   tr_lbs,#.reset_index(drop=True),
                        "lgb_mint": tr_lmint,
                        "ets_base": tr_ebs,
                        "ets_bu":   tr_ebu,
                        "ets_td":   tr_ebs,#.reset_index(drop=True),
                        "ets_mint": tr_emint})
allfcst.to_pickle("721fcsts_base2cohe.pkl")
allfitt.to_pickle("721fitts_base2cohe.pkl")

In [ ]:
allfcst

In [69]:
tr_lbu

0          37929.870202
1          36220.831410
2          34798.572205
3          34825.058046
4          39600.392103
               ...     
1199515        0.177103
1199516        0.186536
1199517        0.174125
1199518        0.242970
1199519        0.142334
Name: LGBMRegressor/BottomUp, Length: 1199520, dtype: float64

In [27]:
import pandas as pd
allfcst = pd.read_pickle("721fcsts_base2cohe.pkl")
allfitt = pd.read_pickle("721fitts_base2cohe.pkl")

In [28]:
allfcst

,lgb_base,lgb_bu,lgb_td,lgb_mint,ets_base,ets_bu,ets_td,ets_mint
0,5.689794,6.307602,5.689794,5.288739,6.747322,6.392221,6.747322,6.949374
1,4.679130,5.639605,4.679130,4.605017,6.525307,6.303542,6.525307,6.231106
2,4.798928,5.449359,4.798928,4.613737,6.398088,6.313164,6.398088,6.544447
3,5.980217,6.054023,5.980217,5.405183,6.448675,6.259084,6.448675,6.743307
4,5.048564,5.671963,5.048564,5.003933,5.990921,6.079106,5.990921,6.804783
...,...,...,...,...,...,...,...,...
1195203,0.177103,0.177103,0.108058,0.088706,0.076043,0.076043,0.083374,0.031334
1195204,0.186536,0.186536,0.105750,0.093228,0.074525,0.074525,0.087415,0.068989
1195205,0.174125,0.174125,0.095396,0.065864,0.074324,0.074324,0.079272,0.095974
1195206,0.242970,0.242970,0.180732,0.142686,0.073504,0.073504,0.081925,0.121862


In [29]:
allfitt

,lgb_base,lgb_bu,lgb_td,lgb_mint,ets_base,ets_bu,ets_td,ets_mint
0,10.427386,31.892026,10.427386,6.554925,4.361950,7.949644,4.361950,6.269733
1,6.017043,18.424095,6.017043,8.308634,5.869864,8.418413,5.869864,7.040606
2,2.831026,5.822718,2.831026,3.816622,7.009515,8.708042,7.009515,7.274240
3,3.167066,5.835477,3.167066,4.299868,7.721546,8.858576,7.721546,7.508198
4,3.511534,4.095703,3.511534,4.408462,8.318248,9.023244,8.318248,8.263259
...,...,...,...,...,...,...,...,...
81658313,0.125571,0.125571,0.125571,0.028646,0.079155,0.079155,0.079155,0.017801
81658314,0.134732,0.134732,0.134732,0.032594,0.078329,0.078329,0.078329,0.041732
81658315,0.139261,0.139261,0.139261,0.036976,0.077508,0.077508,0.077508,0.073077
81658316,0.150921,0.150921,0.150921,0.037568,0.076693,0.076693,0.076693,0.069677
